In [4]:
%pip install sentence-transformers


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install wikipedia-api
%pip install numpy
%pip install scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

/Users/sseadmin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sseadmin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code.

In [15]:
model.max_seq_length = 256

In [16]:
from wikipediaapi import Wikipedia
import numpy as np

In [17]:
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Hayao_Miyazaki').text
paragraphs = doc.split('\n\n')
paragraphs

["Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. He co-founded Studio Ghibli and serves as its honorary chairman. Over the course of his career, Miyazaki has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City, Miyazaki expressed interest in manga and animation from an early age. He joined Toei Animation in 1963, working as an inbetween artist and key animator on films like Gulliver's Travels Beyond the Moon (1965), Puss in Boots (1969), and Animal Treasure Island (1971), before moving to A-Pro in 1971, where he co-directed Lupin the Third Part I (1971–1972) alongside Isao Takahata. After moving to Zuiyō Eizō (later Nippon Animation) in 1973, Miyazaki worked as an animator on World Masterpiece Theater and directed the television

In [18]:
paragraphs = [p for p in paragraphs if len(p.strip()) > 50]

In [19]:
def batch_encode(model, texts, batch_size=8):
    """
    Encode texts in smaller batches to manage memory usage.
    Using a smaller batch size (8) since Wikipedia paragraphs can be quite long.
    """
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, normalize_embeddings=True)
        embeddings.append(batch_embeddings)
        print(f"Processed batch {i//batch_size + 1}/{len(texts)//batch_size + 1}")
    return np.vstack(embeddings)

In [20]:
docs_embed = batch_encode(model, paragraphs)

Processed batch 1/3
Processed batch 2/3
Processed batch 3/3


In [22]:
docs_embed[0]

array([ 1.34032760e-02, -7.74280820e-03,  4.58824933e-02, -1.98609922e-02,
       -3.86114838e-03, -3.21857445e-02,  6.22408018e-02,  3.73538993e-02,
        3.86079475e-02, -4.90618031e-03,  1.42278895e-02, -1.19451247e-03,
        2.53896657e-02,  6.07858459e-03, -2.19852775e-02,  1.25033082e-02,
        2.70423349e-02, -5.60396314e-02, -2.55320650e-02, -4.09074761e-02,
        7.27660768e-03,  2.98580108e-03, -2.13868972e-02,  2.82316003e-02,
       -1.19781317e-02, -2.77474727e-02, -4.17075269e-02, -2.90746931e-02,
       -1.95563529e-02, -8.96699447e-03,  6.38738042e-03, -1.86518729e-02,
        4.09100810e-03,  6.13065669e-03, -2.13458277e-02, -5.50797805e-02,
       -3.80774550e-02,  5.83232827e-02, -1.12530757e-02, -2.47375034e-02,
       -7.71719590e-02,  2.66230572e-03, -3.43225971e-02, -2.15319190e-02,
       -4.01763208e-02,  1.73771363e-02,  1.54944388e-02, -4.89915572e-02,
        1.61564071e-03, -1.86621826e-02, -2.38290280e-02, -1.72906159e-03,
        1.60046425e-02, -

In [23]:
query = "What was Studio Ghibli's first film?"
query_embed = model.encode(query, normalize_embeddings=True)

In [24]:
query_embed

array([-1.22321723e-02, -5.18364273e-02,  5.58609441e-02,  5.67600830e-03,
       -3.86000834e-02, -2.17113458e-02,  3.93825583e-02, -4.23708186e-02,
        1.39679695e-02,  4.74408567e-02, -3.23696397e-02,  1.79127865e-02,
        4.12159413e-03, -1.53939361e-02,  8.31125677e-03, -2.84750368e-02,
        2.67930515e-03, -1.63200554e-02, -3.34369577e-02, -6.75709471e-02,
        2.90005468e-02, -3.50315124e-02, -4.87624370e-02,  1.16035296e-03,
        4.17281035e-03, -4.17729095e-02, -2.33330820e-02, -8.61279853e-03,
        8.41932930e-03,  4.97628516e-03,  6.89192265e-02, -2.27357764e-02,
       -5.31997979e-02,  2.10386254e-02,  3.91372619e-03, -2.11357139e-02,
       -9.22846701e-03,  5.12784347e-02,  1.86153911e-02, -2.57664789e-02,
       -7.98291340e-02,  3.69598530e-02, -4.80721295e-02, -2.59862188e-02,
       -4.76735272e-02, -2.96224430e-02,  7.33324001e-03, -1.63014270e-02,
       -8.29608366e-03,  1.73911471e-02, -2.04304028e-02, -5.83358519e-02,
        1.20177064e-02, -

In [25]:
similarities = np.dot(docs_embed, query_embed.T)

In [26]:
similarities.shape

(20,)

In [27]:
print(similarities)

[0.55563164 0.44476312 0.43233624 0.4505477  0.42667466 0.6631853
 0.51291597 0.5453055  0.46211275 0.528572   0.3916157  0.44915104
 0.43009788 0.42661124 0.41641366 0.4096353  0.43716428 0.39681458
 0.19419621 0.46501273]


In [28]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [29]:
top_3_idx

[5, 0, 7]

In [30]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]
most_similar_documents

['Studio Ghibli\nEarly films (1985–1995)\nFollowing the success of Nausicaä of the Valley of the Wind, Miyazaki and Takahata founded the animation production company Studio Ghibli on June 15, 1985, as a subsidiary of Tokuma Shoten, with offices in Kichijōji designed by Miyazaki. Miyazaki named the studio after the Caproni Ca.309 and the Italian word meaning "a hot wind that blows in the desert"; the name had been registered a year earlier. Suzuki worked for Studio Ghibli as producer, joining full-time in 1989, while Topcraft\'s Tōru Hara became production manager; Suzuki\'s role in the creation of the studio and its films has led him to being occasionally named a co-founder, and Hara is often viewed as influential to the company\'s success. Yasuyoshi Tokuma, the founder of Tokuma Shoten, was also closely related to the company\'s creation, having provided financial backing. Topcraft had been considered as a partner to produce Miyazaki\'s next film, but the company went bankrupt in 1985

In [31]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Studio Ghibli Early films (1985–1995) Following the success of Nausicaä of the Valley of the Wind,
Miyazaki and Takahata founded the animation production company Studio Ghibli on June 15, 1985, as a
subsidiary of Tokuma Shoten, with offices in Kichijōji designed by Miyazaki. Miyazaki named the
studio after the Caproni Ca.309 and the Italian word meaning "a hot wind that blows in the desert";
the name had been registered a year earlier. Suzuki worked for Studio Ghibli as producer, joining
full-time in 1989, while Topcraft's Tōru Hara became production manager; Suzuki's role in the
creation of the studio and its films has led him to being occasionally named a co-founder, and Hara
is often viewed as influential to the company's success. Yasuyoshi Tokuma, the founder of Tokuma
Shoten, was also closely related to the company's creation, having provided financial backing.
Topcraft had been considered as a partner to produce Mi

In [32]:
query = "What was Studio Ghibli's first film?"
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [33]:
import requests
import json

In [34]:
def create_ollama_completion(prompt, model="llama3.2", temperature=0.5):
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except requests.exceptions.RequestException as e:
        print(f"Error making request to Ollama: {e}")
        return None

In [35]:
response = create_ollama_completion(prompt)

In [36]:
response

'The question does not provide enough information to determine what Studio Ghibli\'s first film was. However, it can be inferred that the studio has produced many films throughout its history.\n\nHowever, if you\'re looking for the answer about one of Studio Ghibli\'s early works or the director\'s first solo feature film, I found that "Nausicaa of the Valley of the Wind" (1984) is often considered a landmark work and one of the studio\'s earliest successes.'